In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-09-26 17:36:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-09-26 17:36:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-09-26 17:36:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [12]:
!unzip /home/lenovo/Documents/Hackathon/glove.6B.zip

Archive:  /home/lenovo/Documents/Hackathon/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
#!rm /kaggle/working/glove.6B.zip

rm: cannot remove '/kaggle/working/glove.6B.zip': No such file or directory


In [8]:
!python -m pip install -U gensim

  Using cached https://files.pythonhosted.org/packages/5c/4e/afe2315e08a38967f8a3036bbe7e38b428e9b7a90e823a83d0d49df1adf5/gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b2/97/453576cd8e57cb4f237dbf4dd91097bd26842068661c78a18d61a5ca9f70/smart_open-2.2.0.tar.gz
  Created wheel for smart-open: filename=smart_open-2.2.0-cp37-none-any.whl size=116513 sha256=1dd22be6e9b3af4851ed3f0e08ca976a77d3cad963c9d7fefee4aee96c87d35b
  Stored in directory: /home/lenovo/.cache/pip/wheels/03/04/55/a6c593190f6852d2affb744e398a8fde5b5a63cd0c7e005b83
Successfully built smart-open


In [15]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

glove2word2vec(glove_input_file="/home/lenovo/Documents/Hackathon/glove.6B/glove.6B.100d.txt", word2vec_output_file="gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [16]:
import pandas as pd
import numpy as np
import nltk

In [17]:
# map_dict = {"positive": 4, "negative":0}
# financial_news_data = pd.read_csv("../input/sentiment-analysis-for-financial-news/all-data.csv", encoding="ISO 8859-1", header=None)
# financial_news_data[0] = financial_news_data[0].map(map_dict)
twitter_data = pd.read_csv("/home/lenovo/Documents/Hackathon/archive/training.1600000.processed.noemoticon.csv", encoding="ISO 8859-1", header=None)

In [18]:
print(twitter_data.head())
# print(financial_news_data.head())

   0           1                             2         3                4  \
0  0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   

                                                   5  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  is upset that he can't update his Facebook by ...  
2  @Kenichan I dived many times for the ball. Man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  


In [19]:
twitter_data = twitter_data[[0, 5]]
twitter_data.rename({0:"sentiment", 5:"text"}, axis=1, inplace=True)
# financial_news_data.rename({0:"sentiment", 1:"text"}, axis=1, inplace=True)

# twitter_data = pd.concat([twitter_data, financial_news_data], axis=0)

twitter_data.reset_index(inplace=True)

twitter_data["text"] = twitter_data["text"].str.split(' ').apply(lambda x: ' '.join([k for k in x if not (('http://' in k) or ('.com'  in k) or ('@'  in k) or ('#'  in k))]))
twitter_data["text"] = twitter_data["text"].str.replace(r'[^a-zA-Z\s]','').str.lower()

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
vocab = glove_model.vocab.keys()

In [22]:
#SENTENCE EMBEDDING

import numpy as np


def gen_mean(vals, p):
    p = float(p)
    return np.power(
        np.mean(
            np.power(
                np.array(vals, dtype=complex),
                p),
            axis=0),
        1 / p
    )


operations = dict([
    ('mean', (lambda word_embeddings: [np.mean(word_embeddings, axis=0)], lambda embeddings_size: embeddings_size)),
    ('max', (lambda word_embeddings: [np.max(word_embeddings, axis=0)], lambda embeddings_size: embeddings_size)),
    ('min', (lambda word_embeddings: [np.min(word_embeddings, axis=0)], lambda embeddings_size: embeddings_size)),
    ('p_mean_2', (lambda word_embeddings: [gen_mean(word_embeddings, p=2.0).real], lambda embeddings_size: embeddings_size)),
    ('p_mean_3', (lambda word_embeddings: [gen_mean(word_embeddings, p=3.0).real], lambda embeddings_size: embeddings_size)),
])


def get_sentence_embedding(sentence, embeddings, chosen_operations):
    word_embeddings = []
    for tok in sentence:
        if tok not in vocab:
            continue
        vec = embeddings[tok]
        if vec is not None:
            word_embeddings.append(vec)

    if not word_embeddings:
        sentence_embedding = np.zeros(300)
    else:
        concat_embs = []
        for o in chosen_operations:
            concat_embs += operations[o][0](word_embeddings)
        sentence_embedding = np.concatenate(
            concat_embs,
            axis=0
        )

    return sentence_embedding

In [23]:
def get_embedding(sent):
    return get_sentence_embedding(sent.split(), glove_model, ['min', 'mean', 'max'])

In [24]:
twitter_data = twitter_data[["sentiment", "text"]]
twitter_data["text"] = twitter_data["text"].apply(lambda x: get_embedding(x))
X_train_val = []
for index in range(len(twitter_data)):
    X_train_val.append(twitter_data["text"][index])
    
X_train_val = np.array(X_train_val)

In [25]:
y_train_val = []
for index in range(len(twitter_data)):
    y_train_val.append(twitter_data["sentiment"][index])

In [ ]:
# np.save("X_train.npy", X_train_val)
# np.save("y_train.npy", np.array(y_train_val))

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(twitter_data["text"], twitter_data["sentiment"], test_size=0.25, random_state=33)

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=33)

In [27]:
y_train = np.array(y_train, dtype=np.int16)
y_train[y_train>2] = 1
y_train

array([0, 1, 1, ..., 1, 1, 1], dtype=int16)

In [28]:
import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [29]:
cat_y_train = keras.utils.to_categorical(y_train, 2)

In [1]:
input_shape = (32, 900)
from tensorflow.keras.models import Sequential

def create_model():
    model = Sequential()

    # model.add(layers.Conv2D(filters=6, kernel_size=(3, 1), strides=(1,1), activation='relu', input_shape=input_shape))
    # model.add(layers.AveragePooling2D())

    # model.add(layers.Conv2D(filters=16, kernel_size=(3, 1), strides=(1,1), activation='relu'))
    # model.add(layers.AveragePooling2D())

    model.add(layers.Flatten())
    model.add(layers.Dense(units=900, activation='relu'))
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=32, activation='relu'))
    model.add(layers.Dense(units=2, activation = 'softmax'))
    
    return model


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

NameError: name 'model' is not defined

In [34]:
model.fit(np.array(X_train, dtype=np.float32), cat_y_train, epochs=5, batch_size=32)

Train on 1200000 samples
Epoch 1/5
1200000/1200000 [==============================] - 500s 417us/sample - loss: 0.5400 - accuracy: 0.7223
Epoch 2/5
1200000/1200000 [==============================] - 522s 435us/sample - loss: 0.5126 - accuracy: 0.7434
Epoch 3/5
1200000/1200000 [==============================] - 534s 445us/sample - loss: 0.5028 - accuracy: 0.7501
Epoch 4/5
1200000/1200000 [==============================] - 543s 453us/sample - loss: 0.4960 - accuracy: 0.7546
Epoch 5/5
1200000/1200000 [==============================] - 540s 450us/sample - loss: 0.4910 - accuracy: 0.7578


In [35]:
model.save("twitter_sentiment_100d.h5")

In [36]:
import requests

def NewsFromBBC(): 

    # BBC news api 
    main_url = "https://newsapi.org/v2/everything?"
    parameters = {
    'q': 'big data', # query phrase
    'pageSize': 5,  # maximum is 100
    'apiKey': '38c5b69ec17c41b5aa61f1d6467f18ef' # your own API key
    }
    # fetching data in json format 
    open_bbc_page = requests.get(main_url, params=parameters).json() 

    # getting all articles in a string article 
    article = open_bbc_page["articles"] 

    # empty list which will
    # contain all trending news 
    results = [] 

    for ar in article: 
        results.append(ar["title"]) 

#     for i in range(len(results)): 

#         # printing all trending news 
#         print(i + 1, results[i])

    return results

# Driver Code 
# if name == 'main': 

    # function call 
results = NewsFromBBC() 
print(results)

['Drive predictable B2B revenue growth with insights from big data and CDPs', 'Daily Crunch: Shopify confirms data breach', 'How to Piss Off Advertisers With Your iOS 14 Settings', 'Transposit scores $35M to build data-driven runbooks for faster disaster recovery', 'Big tech has 2 elephants in the room: Privacy and competition']


In [39]:
df = pd.DataFrame(results) 
df

,0
0,Drive predictable B2B revenue growth with insi...
1,Daily Crunch: Shopify confirms data breach
2,How to Piss Off Advertisers With Your iOS 14 S...
3,Transposit scores $35M to build data-driven ru...
4,Big tech has 2 elephants in the room: Privacy ...


In [40]:
df[0] = df[0].str.split(' ').apply(lambda x: ' '.join([k for k in x if not (('http://' in k) or ('.com'  in k) or ('@'  in k) or ('#'  in k))]))
df[0] = df[0].str.replace(r'[^a-zA-Z\s]','').str.lower()

In [41]:
df

,0
0,drive predictable bb revenue growth with insig...
1,daily crunch shopify confirms data breach
2,how to piss off advertisers with your ios set...
3,transposit scores m to build datadriven runboo...
4,big tech has elephants in the room privacy an...


In [42]:
test_data = df[0].apply(lambda x: get_embedding(x))
X_test = []
for index in range(len(test_data)):
    X_test.append(test_data[index])
    
X_test = np.array(X_test)

In [44]:
classes = model.predict(X_test)

In [45]:
np.argmax(classes, axis=1)

array([1, 0, 1, 1, 1])

In [49]:
for i in range(len(df)):
    print(df[0][i])

drive predictable bb revenue growth with insights from big data and cdps
daily crunch shopify confirms data breach
how to piss off advertisers with your ios  settings
transposit scores m to build datadriven runbooks for faster disaster recovery
big tech has  elephants in the room privacy and competition


In [50]:
def pre_embedd(results):
    df = pd.DataFrame(results)
    df[0] = df[0].str.split(' ').apply(lambda x: ' '.join([k for k in x if not (('http://' in k) or ('.com'  in k) or ('@'  in k) or ('#'  in k))]))
    df[0] = df[0].str.replace(r'[^a-zA-Z\s]','').str.lower()
    test_data = df[0].apply(lambda x: get_embedding(x))
    X_test = []
    for index in range(len(test_data)):
        X_test.append(test_data[index])
    
    X_test = np.array(X_test)
    return X_test
    

In [57]:
def predict1(X_test):
    classes = model.predict(X_test)
    return np.argmax(classes, axis=1)
    
    

In [56]:
# from sklearn.pipeline import Pipeline
# pipeline = Pipeline(steps= [('embedding', pre_embedd(results)),
#                             ('model', predict(X_test))])

In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = keras.models.load_model("twitter_sentiment_100d.h5")
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    results = NewsFromBBC() 
    X_test = pre_embedd(results)
    predicted = predict1(X_test)
    
    
    
    

#     int_features = [int(x) for x in request.form.values()]
#     final_features = [np.array(int_features)]
#     prediction = model.predict(final_features)

#     output = round(prediction[0], 2)

    return render_template('index.html', prediction_text='Sales should be $ {}'.format(predicted))

#@app.route('/results',methods=['POST'])
# def results():

#     data = request.get_json(force=True)
#     prediction = model.predict([np.array(list(data.values()))])

#     output = prediction[0]
#     return jsonify(output)

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
y_val = np.array(y_val, dtype=np.int16)
y_val[y_val>2.5] = 1

cat_y_val = keras.utils.to_categorical(y_val, 3)

In [ ]:
loss_and_metrics = model.evaluate(X_val, cat_y_val, batch_size=128)

In [ ]:
loss_and_metrics

In [ ]:
news_data = pd.read_csv("/kaggle/input/million-headlines/abcnews-date-text.csv")

In [ ]:
news_data["headline_text"][:32]

In [ ]:
test_data = news_data["headline_text"][:32].apply(lambda x: get_embedding(x))
X_test = []
for index in range(len(test_data)):
    X_test.append(test_data[index])
    
X_test = np.array(X_test)

In [ ]:
classes = model.predict(X_test, batch_size=32)

In [ ]:
np.argmax(classes, axis=1)

In [ ]:
for 
news_data["headline_text"][2]

In [ ]:
np.argmax(model.predict(X_val[:32], batch_size=32), axis=1)

In [ ]:
twitter_data = pd.read_csv("../input/sentiment140/training.1600000.processed.noemoticon.csv", encoding="ISO 8859-1", header=None, nrows=32)

In [ ]:
twitter_data = twitter_data[[0, 5]]
twitter_data.rename({0:"sentiment", 5:"text"}, axis=1, inplace=True)

twitter_data.reset_index(inplace=True)

twitter_data["text"] = twitter_data["text"].str.split(' ').apply(lambda x: ' '.join([k for k in x if not (('http://' in k) or ('.com'  in k) or ('@'  in k) or ('#'  in k))]))
twitter_data["text"] = twitter_data["text"].str.replace(r'[^a-zA-Z\s]','').str.lower()

In [ ]:
for text in twitter_data["text"]:
    print(text)
    print("\n")

In [ ]:
twitter_data["text"][:32].apply(lambda x: get_embedding(x))
X_test = []
for index in range(len(test_data)):
    X_test.append(test_data[index])
    
X_test = np.array(X_test)

In [ ]:
np.argmax(model.predict(X_val[:32], batch_size=32), axis=1)